# RecSeats

## Individual-level component

Main file for running all individual-level choice models (LR, SVC, GBT, RF) on Locational choice experiment data.

**Author of the code:** Anon.


#### Librairies import

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import copy

import csv
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import make_scorer

import os

#### Function from other files import

In [ ]:
from src.preprocessing.data_loading import load_data_matrix
from src.preprocessing.compute_features import Feature_Pipeline
from src.utils import Params

from src.model.user_specific import *
from src.metrics import *
from src.room_transformation import *

from src.visualisation.plot_example import * 

#### File setting

Changing the `IND_FILE` value allows to change the studied dataset 

In [ ]:
# Index of the file to study:
IND_FILE = 0

params_list = ["study2.json", #0
               "study4_CF_FC.json", #1
              ]

path_data = "./data/Blanchard/"

params_file = Params(path_data + "parameters/" + params_list[IND_FILE])

print("INSAMPLE : ", params_file.csv_train)
print("HOLDOUT : ", params_file.csv_valid)
print("ROOM SIZE : ", params_file.room_size)
print("PADDING : ", params_file.padding)
print("PAIRS OF SEATS : ", params_file.is_couple)


#### Data Loading

In [ ]:
try:
    train_inputs = np.load(path_data+"numpy/"+params_file.dataloader_train+"_inputs.npy", allow_pickle=True)
    train_outputs = np.load(path_data+"numpy/"+params_file.dataloader_train+"_outputs.npy", allow_pickle=True)
    
except FileNotFoundError:
    print("Creating data matrix for train set...", end="")
    train_inputs, train_outputs = load_data_matrix(path = path_data + params_file.csv_train, 
                                                   room_size = params_file.room_size, 
                                                   padding = 0,
                                                   verbose = False,
                                                   to_tensor = 0,
                                                   is_wso = params_file.is_wso
                                                   )
    np.save(path_data+"numpy/"+params_file.dataloader_train+"_inputs.npy", train_inputs)
    np.save(path_data+"numpy/"+params_file.dataloader_train+"_outputs.npy", train_outputs)
    print("Done.")


try:
    valid_inputs = np.load(path_data+"numpy/"+params_file.dataloader_valid+"_inputs.npy", allow_pickle=True)
    valid_outputs = np.load(path_data+"numpy/"+params_file.dataloader_valid+"_outputs.npy", allow_pickle=True)
    
except FileNotFoundError:
    print("Creating data matrix for valid set...", end="")
    valid_inputs, valid_outputs = load_data_matrix(path = path_data + params_file.csv_valid, 
                                                   room_size = params_file.room_size, 
                                                   padding = 0,
                                                   verbose = False,
                                                   to_tensor = 0,
                                                   is_wso = params_file.is_wso
                                                   )
    np.save(path_data+"numpy/"+params_file.dataloader_valid+"_inputs.npy", valid_inputs)
    np.save(path_data+"numpy/"+params_file.dataloader_valid+"_outputs.npy", valid_outputs)
    print("Done.")


If the file contains pairs of seats, then the transformation to predict the seat on the left is applied on the dataset:

In [ ]:
if params_file.is_couple:
    for i in range(train_inputs.shape[0]):
        for j in range(train_inputs.shape[1]):
            train_inputs[i][j] = keep_left_seat(train_inputs[i][j])
        for j in range(valid_inputs.shape[1]):
            valid_inputs[i][j] = keep_left_seat(valid_inputs[i][j])


### Training and Evaluation

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
params_parametric = Params("src/model/parameters/params_user_specific.json")

pipeline = Feature_Pipeline(params_parametric)

model = Recommendation(params_parametric, pipeline)

scorer = make_scorer(top_n_accuracy)
scorer_3 = make_scorer(top_n_accuracy, n = 3)
scorer_5 = make_scorer(top_n_accuracy, n = 5)
scorer_10 = make_scorer(top_n_accuracy, n = 10)
scorer_20 = make_scorer(top_n_accuracy, n = 20)
evals_scorer = [scorer, scorer_3, scorer_5]

beta_list, train_acc, valid_acc, cross_acc = model.train_evaluate(train_inputs, train_outputs,
                                                                  evals_scorer = evals_scorer,
                                                                  test_X = valid_inputs,
                                                                  test_Y = valid_outputs,
                                                                  verbose = False)
np.save("save/beta_values.npy", beta_list)

### L1 Loss computation :

In [ ]:
train_sizes = np.asarray([[[len(train_inputs[k][i]), len(train_inputs[k][i][0])] for i in range(len(train_inputs[k]))] 
                          for k in range(len(train_inputs))])
valid_sizes = np.asarray([[[len(valid_inputs[k][i]), len(valid_inputs[k][i][0])] for i in range(len(valid_inputs[k]))] 
                          for k in range(len(valid_inputs))])
l1_loss = []

for i in range(len(train_inputs)):
    train_X, train_Y = model.pipeline.compute_feature(train_inputs[i], train_outputs[i])
    valid_X, valid_Y = model.pipeline.compute_feature(valid_inputs[i], valid_outputs[i])

    model.fit(train_X, train_Y)

    l1_loss.append(weighted_l1(model, valid_X, valid_Y, valid_sizes[i]))
    print("\r{}/{}".format(i + 1, len(train_inputs)), end="")


In [ ]:
weighted_l1(model, valid_X, valid_Y, valid_sizes[i])

In [ ]:
np.nanmean(l1_loss)